# 4. Dimensional Reduction
Compiled by [Morgan Williams](mailto:morgan.williams@csiro.au) for C3DIS 2018 

Traditional analysis has utilised the similarities in physiochemical properties between geochemical components as a (generally valid) method of dimensional reduction, but excludes data which may otherwise be useful indicators of subtle differences.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA

sys.path.insert(0, './src')
from compositions import *
from geochem import *

np.random.seed(1)

In [2]:
fndir = r'data/examples/'
fns = ['boxite.xlsx', 'coxite.xlsx', 'hongite.xlsx', 'kongite.xlsx']
dfs = [pd.read_excel(os.path.join(fndir, f)) for f in fns]
[setattr(df, 'label', fns[ix]) for ix, df in enumerate(dfs)]
B, C, H, K = dfs

In [3]:
print(B)

        A     B     C     D    E  Depth
B1   43.5  25.1  14.7  10.0  6.7      1
B2   41.1  27.5  13.9   9.5  8.0      2
B3   41.5  20.1  20.6  11.1  6.7      3
B4   33.9  37.8  11.1  11.5  5.7      4
B5   46.5  16.0  15.6  14.3  7.6      5
B6   45.3  19.4  14.8  13.5  9.3      6
B7   33.2  25.2  15.2  17.1  9.3      7
B8   40.8  15.1  21.7  14.6  7.8      8
B9   33.0  30.8  15.1  12.9  8.2      9
B10  28.2  38.6  12.1  14.1  6.9     10
B11  33.9  31.5  15.4  12.0  7.2     11
B12  48.7  19.3  13.4  10.7  7.9     12
B13  37.8  37.1  10.4   8.6  6.1     13
B14  42.0  26.6  13.7  10.5  7.2     14
B15  44.2  26.5  12.9   9.6  6.8     15
B16  39.7  23.2  20.6  10.2  6.3     16
B17  39.3  28.1  13.0  13.6  6.0     17
B18  34.1  26.7  13.6  17.0  8.6     18
B19  36.2  35.3  11.2  11.9  5.4     19
B20  39.5  36.0   9.4   8.4  6.7     20
B21  39.5  22.5  18.7  11.4  7.9     21
B22  33.0  33.5  17.7   9.8  6.0     22
B23  42.3  16.6  16.9  17.0  7.2     23
B24  39.9  19.0  13.4  21.3  6.4     24


## Principal Component Analysis

## Measures of Geochemical Similarity

## Mixtures

#### Polytypic Vector Analysis
Reproduce analysis of Szymanski et al (2013): Evaluating Complex Magma Mixing via Polytopic Vector Analysis (PVA) in the Papagayo Tuff, Northern Costa Rica: Processes that Form Continental Crust